# Ejercicio 1 –Spark Core

In [1]:
# Import findpsark
import findspark

# Configure the environment
findspark.init()

# Import the Spark components required for the context creation
from pyspark import SparkConf, SparkContext

# Configure and create the context
conf = SparkConf()
conf = conf.setAppName('mds-session')
conf = conf.setMaster('local[*]')
sc = SparkContext.getOrCreate(conf=conf)

22/04/04 21:09:40 WARN Utils: Your hostname, mdsuser resolves to a loopback address: 127.0.1.1; using 192.168.242.129 instead (on interface ens33)
22/04/04 21:09:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 21:09:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# load albums
def parseAlbum(line):
    line = line.split('\t')
    return {
        'id': line[0],
        'title': line[1]
    }

albums = sc.textFile('./data/albums.tsv')
albums = albums.map(parseAlbum)
albums.first()

{'id': '300848', 'title': 'Fear Itself'}

In [3]:
# load artists
def parseArtist(line):
    line = line.split('\t')
    return {
        'id': line[0],
        'name': line[1],
        'hotness': float(line[2]) if line[2] != 'NA' else -9999,
        'familiarity': float(line[3]) if line[3] != 'NA' else -9999,
        'location': line[4]
    }
artists = sc.textFile('./data/artists.tsv')
artists = artists.map(parseArtist)
artists.first()

{'id': 'AR009211187B989185',
 'name': 'Carroll Thompson',
 'hotness': 0.297404803,
 'familiarity': 0.396817844,
 'location': 'Not available'}

In [4]:
# load songs
def parseSong(line):
    line = line.split('\t')
    return {
        'id': line[0], 
        'title': line[1], 
        'year': int(line[2]) if line[2] != 'NA' else -9999,
        'hotness': float(line[3]) if line[3] != 'NA' else -9999,
        'id_artist': line[4],
        'id_album': line[5],
        'duration': float(line[6]) if line[6] != 'NA' else -9999,
        'end_of_fade_on': float(line[7]) if line[7] != 'NA' else -9999,
        'start_of_fade_out': float(line[8]) if line[8] != 'NA' else -9999,
        'tempo': float(line[9]) if line[9] != 'NA' else -9999,
        'time_signature': int(line[10]) if line[10] != 'NA' else -9999,
        'key': int(line[11]) if line[11] != 'NA' else -9999,
        'loudness': float(line[12]) if line[12] != 'NA' else -9999,
        'mode': int(line[13]) if line[13] != 'NA' else -9999,
        'style': line[14]
    }

songs = sc.textFile('./data/songs.tsv')
songs = songs.map(parseSong)
songs.first()

{'id': 'SOAAAQN12AB01856D3',
 'title': 'Campeones De La Vida',
 'year': 0,
 'hotness': 0.427446572,
 'id_artist': 'ARAMIDF1187FB3D8D4',
 'id_album': '346208',
 'duration': 153.36444,
 'end_of_fade_on': 0.235,
 'start_of_fade_out': 142.518,
 'tempo': 120.051,
 'time_signature': 4,
 'key': 4,
 'loudness': -8.76,
 'mode': 0,
 'style': 'pop rock'}

### 1.¿Cuál es el estilo más lento(tempo) en media?

In [5]:
def prepare_mean(pair1, pair2):
    summatory = pair1[0] + pair2[0] if pair2[0] != -9999 else pair1[0]
    num_elem = pair1[1] + pair2[1] if pair2[0] != -9999 else pair[1]
    return (summatory, num_elem)

def calculate_mean(pair):
    style = pair[0]
    mean = pair[1][0]/pair[1][1]
    return (style, mean)

songs.map(lambda song: (song['style'], (song['tempo'],1))).reduceByKey(prepare_mean).\
    map(calculate_mean).sortBy(lambda pair: pair[1]).first()

('rebetika', 47.447)

### 2.¿Cuales  son  los  5  artistas,  ubicados  en  UK(cualquier  territorio  de  UK),  con mayor número de canciones en escala menor (mode = 1)?

In [6]:
def filter_UK_mode(pair):
    return  ("UK" in pair[1] or
            "United Kingdom" in pair[1] or
            "England" in pair[1] or
            "Wales" in pair[1] or
            "Scotland" in pair[1] or
            "Northern Ireland" in pair[1]) and pair[2] == 1


songs_artist_mode = songs.map(lambda song: (song['id_artist'], song['mode']))
artist_location = artists.map(lambda artist: (artist['id'], (artist['location'], artist['name'])))

songs_artist_mode.join(artist_location).map(lambda pair: (pair[1][1][1], pair[1][1][0], pair[1][0])).\
    filter(filter_UK_mode).map(lambda info: (info[0], 1)).reduceByKey(lambda elem1, elem2: elem1 + elem2).\
    sortBy(lambda pair: pair[1], ascending = False).take(6)



[('SNOWPATROL', 11),
 ('Phil Collins', 9),
 ('The Rolling Stones', 7),
 ('Radiohead', 7),
 ('Seal', 6),
 ('Bad Company', 6)]

### 3.Desde  1970  hasta  hoy,  ¿las  canciones  son  más  rápidas  (tempo),  altas (loudness)  y cortas  (duration)  en  media?  Ordena  los  resultados  por  año ascendente.

In [7]:
def prepare_mean_2(pair1, pair2):
    summatory_tempo = pair1[0][0] + pair2[0][0] if pair2[0][0] != -9999 else pair1[0][0]
    num_elem_tempo = pair1[0][1] + pair2[0][1] if pair2[0][0] != -9999 else pair1[0][1]
    
    summatory_loudness = pair1[1][0] + pair2[1][0] if pair2[1][0] != -9999 else pair1[1][0]
    num_elem_loudness = pair1[1][1] + pair2[1][1] if pair2[1][0] != -9999 else pair1[1][1]
    
    summatory_duration = pair1[2][0] + pair2[2][0] if pair2[2][0] != -9999 else pair1[2][0]
    num_elem_duration = pair1[2][1] + pair2[2][1] if pair2[2][0] != -9999 else pair1[2][1]
    
    return ((summatory_tempo, num_elem_tempo), (summatory_loudness, num_elem_loudness), (summatory_duration, num_elem_duration))

def calculate_mean_2(elem):
    year = elem[0]
    mean_tempo = elem[1][0][0]/elem[1][0][1]
    mean_loudness = elem[1][1][0]/elem[1][1][1]
    mean_duration = elem[1][2][0]/elem[1][2][1]
    return (year, mean_tempo, mean_loudness, mean_duration)

songs.filter(lambda song: song['year'] >= 1970).map(lambda song: (song['year'], ((song['tempo'],1), (song['loudness'],1), (song['duration'],1)))).\
    reduceByKey(prepare_mean_2).map(calculate_mean_2).sortBy(lambda elem: elem[0]).take(20)

[(1970, 121.34628571428571, -11.92847619047619, 231.42578619047623),
 (1971, 136.16196, -12.153000000000002, 259.55428919999997),
 (1972, 129.17204166666667, -11.719291666666665, 238.54539750000004),
 (1973, 116.356125, -11.711541666666667, 294.16444416666667),
 (1974, 125.08609090909088, -10.670681818181817, 239.49134636363635),
 (1975, 125.41183333333333, -11.249541666666666, 277.4406354166667),
 (1976, 137.26139999999998, -11.6584, 210.9940493333333),
 (1977, 139.33685714285713, -11.820114285714284, 255.30692800000003),
 (1978, 134.38385, -10.1125, 247.85456749999997),
 (1979, 137.51694444444445, -11.879083333333334, 226.0566886111111),
 (1980, 126.89337499999999, -11.09853125, 210.438730625),
 (1981, 127.96074999999999, -11.570444444444446, 211.69224499999999),
 (1982, 125.14522, -11.348739999999998, 251.23742419999996),
 (1983, 126.14368085106385, -12.30287234042553, 237.46428063829788),
 (1984, 127.95625, -12.015031250000002, 239.31873),
 (1985, 132.86585294117648, -10.3137352941

### 4.¿Cuál es el estilo que menos abusa de los efectos de fade in y fade out (mayor número de segundos desde inicio al final del fade in más desde el inicio del fade out al final de la canción?

In [8]:
def calculate_fade_and_add_cont(pair):
    style = pair[0]
    fade = pair[1][0] + (pair[1][1] - pair[1][2])
    return (style, (fade, 1))
    
    
def prepare_mean_3(pair1, pair2):
    summatory = pair1[0] + pair2[0]
    num_elem = pair1[1] + pair2[1]
    return (summatory, num_elem)

def calculate_mean_3(pair):
    style = pair[0]
    mean = pair[1][0]/pair[1][1]
    return (style, mean)


songs.map(lambda song: (song['style'], (song['end_of_fade_on'], song['duration'], song['start_of_fade_out']))).\
    map(calculate_fade_and_add_cont).reduceByKey(prepare_mean_3).map(calculate_mean_3).\
    sortBy(lambda pair: pair[1]).take(5)

[('power violence', -0.0003299999999981651),
 ('christian hardcore', -0.00019500000000505224),
 ('east coast hip hop', 0.00011999999998124622),
 ('argentine rock', 0.0005099999999913507),
 ('melodic trance', 0.103)]

Con Spark Queries habíamos visto que eran posibles valores negativos.

### 5.¿Cual es la canción menos popular (hotness) de los 5 artistas más populares (hotness)?

In [9]:
def select_less_popular(pair1, pair2):
    if (pair1[1] == -9999):
        return pair2
    elif (pair2[1] == -9999):
        return pair1
    elif (pair2[1] < pair1[1]):
        return pair2
    else: 
        return pair1

# como son únicamente 5 artistas vamos a traerlos a memoria
popular_artists = artists.sortBy(lambda artist: artist['hotness'], ascending = False).map(lambda artist: artist['id']).take(5)

songs.map(lambda song: (song['id_artist'], (song['title'], song['hotness']))).\
    filter(lambda song: song[0] in popular_artists).reduceByKey(select_less_popular).map(lambda pair: pair[1]).take(5)

[('Magic', 0.508602172),
 ('Skit #2 (Kanye West/Late Registration)', 0.7801197),
 ('Da Funk', 0.8622545),
 ('The Way I Loved You', 0.853828893),
 ('Speed Of Sound (Live)', 0.454042766)]

In [10]:
sc.stop()